# Seminar Quantitativ Microbiology 
# Metabolite Production with Strain Engineering

## Introduction


For a overview on Jupyter notebooks read [this review](https://www.nature.com/articles/d41586-018-07196-1). 

---

## Tutorial Steps
  * Set up of Python environment
    * Basic libraries(sys, pandas, numpy, matplotlib, zipfile, cobrapy)
  * Analysis of Genome Scale Metabolic Model
    * Retrieval of GSMM for *P. pastoris*
    * Flux variability of exchange reactions
    * Minimal medium composition
  * Experimental growth rate reproduction
    * Familiarizing with biomass composition reactions
    * Defining functions for correct biomass equation switch
    * Data retrieval
    * Simulation loop
    * Graphical output


## Set-up compute environment

Before we can analyse GSMM, we have adjust the python environment that it integrates the cobrapy toolbox and downloading the GSMM.

### Basic Python libraries 
Some libraries that facilitate data manipulation

In [ ]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model, write_sbml_model, load_matlab_model

# loading escher for metabolic network visualization
# import escher
# from escher import Builder
# from time import sleep
# escher.rc['never_ask_before_quit'] = True
# list of available maps
# print(escher.list_available_maps())

# loading Memote, quality assessment of GSMM
# from memote import test_model, snapshot_report

from BioLabSim.ModuleVirtualOrganism.HostSetup import Host, Strain

print('System ready')

In [ ]:
myHost = Host('Pput', Metabolism=True)
myHost.add_Strain('Mut1')
myHost.show_BiotechSetting()
print('Wild type growth rate: {:.2f}'.format(myHost.WT.info_Objective))

In [ ]:
from BioLabSim.ModuleVirtualOrganism.Metabolism import Help_StrainCharacterizer
RctNewDF = Help_StrainCharacterizer(myHost.var_Host, myHost.WT.info_GenesDF, myHost.WT.info_Genome, myHost.Mut1.info_Genome, myHost.WT.var_Model)
RctNew = RctNewDF[RctNewDF['RctFlag']==True].index.values
print(RctNew)
# For reactions with reduced expression and positive flux: reduce the upper limit,
# For reactions with increased expression and positive flux: increase the lower limit
# for reactions with negative flux the limits are exchanged.
FluxPos = RctNew[tuple([RctNewDF.loc[RctNew, 'RefFlux']>0])]
FluxNeg = RctNew[tuple([RctNewDF.loc[RctNew, 'RefFlux']<0])]
# Finding increased and decreased fluxes
FluxInc = RctNew[RctNewDF.loc[RctNew, 'NewExpr'].values / RctNewDF.loc[RctNew, 'RefExpr'].values>1]
FluxDec = RctNew[RctNewDF.loc[RctNew, 'NewExpr'].values / RctNewDF.loc[RctNew, 'RefExpr'].values<1]
print('Positive: {}, Negative: {}'.format(FluxPos,FluxNeg))
print('Increase: {}, Decrease: {}'.format(FluxInc,FluxDec))

In [ ]:
from BioLabSim.ModuleMeasureOrganism.Sequencing import measure_BaseCompare
from BioLabSim.ModuleMeasureOrganism.Fluxes import measure_EnzymeLevel1
# myDiff = measure_BaseCompare(myHost.WT.info_Genome, myHost.Mut1.info_Genome)
a,b,c = measure_EnzymeLevel1(myHost, myHost.Mut1)
# print(myDiff)
print(b,c)

In [ ]:
myHost.WT.info_GenesDF


In [ ]:
FluxDiff = myHost.Mut1.info_GenesDF['Fluxes']/myHost.WT.info_GenesDF['Fluxes']
np.nan_to_num(FluxDiff, nan=1, posinf=1, neginf=1)

In [ ]:
with open('WT-Seq.fasta', 'w') as file:
    file.write('>WT_FullSeq\n{}'.format(myHost.WT.info_Genome))

## Promoter identification with SAPPHIRE